# Create XML Metadata from Table of Metadata
* Table of metadata is created at same time as tabular (i.e. csv or xlsx) data. 
* Metadata is useful to both scientists during life of project, as well as for future users and data release :+1:


In [1]:
'''
  import external libraries
'''
import glob
import pandas as pd
from lxml import etree
import os
import datetime

'''
 import pymdwizard libraries
'''
import sys
path_to_pymdwizard=r"C:/Users/ehbaker/Documents/Python/scriptsFromOthers/fort-pymdwizard" #path to your pymdwizard local copy
sys.path.append(path_to_pymdwizard)
from pymdwizard.core.xml_utils import XMLRecord, XMLNode

In [2]:
#Import Metadata Table
#substitute path for new table
met_tab_pth=r"Q:\Project Data\GlacierData\DataManagement\CompletedMetTables\MetadataTable_GlacierNameYYYY.MM.DD PitCore_SiteLabel.xlsx" #path to metadata table
met_tab=pd.read_excel(met_tab_pth, sheetname='table_level')
row=met_tab.iloc[0,:] #isolate the first row

### Table-Level Metadata (relevant to all data in table)
* this data is created by the scientist, at the same time they are collating their data 
* first sheet in excel metadata workbook
* each line is for a separate table in the relevant data

In [3]:
met_tab

,table,description,methods,methods_citation,publication_citation,authors,associated_tables,supplemental_info
0,GlacierNameYYYY.MM.DD PitCore_SiteLabel.csv,Raw snow pit and snow core measurements of sam...,"Samples are taken using a variety of samplers,...","Ostream, G., and M. Brugman (1991), Glacier Ma...",NaN,"Emily H. Baker, Christopher J. McNeil, Louis C...",NaN,NaN


### Add External XML Files 
* first (xmlpth) is the full FGDC template, with all fields consistent to your group's data releases pre-filed
* second (attr_numeric_path) is a path to an XML document for a blank 'numeric attribute" part of the FGDC section

In [4]:
#Import XML Structure
#paths
xmlpth=r"C:\Users\ehbaker\Documents\XML_Work\FGDC_template_Full.xml"
attr_numeric_pth=r"C:\Users\ehbaker\Documents\XML_Work\attr_numeric.xml"
attr_factor_path=r"C:\Users\ehbaker\Documents\XML_Work\attr_factor.xml"
category_details_path=r"C:\Users\ehbaker\Documents\XML_Work\category.xml"

In [5]:
#Create XMLRecords and XMLNodes from flat files
metd=XMLRecord(xmlpth)

# Add Table-Level Metadata
* More complex datasets (with multiple tables) could loop through all rows in the table, with "for row in table: ..."

In [6]:
#Add Data Description
metd.metadata.idinfo.descript.abstract.text=row['description'] 

##### We can see this has added the text from the "description" column to our FGDC metadata:

In [7]:
metd.metadata.idinfo.descript

<descript>
  <abstract>Raw snow pit and snow core measurements of sample weight and volume, at specified depth from the surface</abstract>
  <purpose>

  </purpose>
</descript>

#### Add methods, citation, and publication citation

In [8]:
#Add methods (table-level description)
metd.metadata.dataqual.lineage.procstep.procdesc.text=row['methods']

#Add citation for methods used (at table-level)
metd.metadata.dataqual.lineage.procstep.srcused.text=row['methods_citation']
#Add citation for publication that resulted from this data
metd.metadata.dataqual.lineage.procstep.srcprod.text=row['publication_citation']

#### Add authors:
* data entry is in single cell, but each author must be a separate node

In [9]:
#Add authors
auth_node=XMLNode("<origin></origin>")
authors= row.authors.split(',')#list of authors; input in single cell separated by commas
#Set existing first author node to first author (already in base XML, required field)
metd.metadata.idinfo.citation.citeinfo.origin.text=authors[0]
#Add nodes for remainder of authors in list
for author in authors[1:]:
    auth_node.text=author
    metd.metadata.idinfo.citation.citeinfo.add_child(auth_node)

### Examine resulting metadata

In [10]:
metd.metadata.dataqual.lineage.procstep.procdesc.text #for example

'Samples are taken using a variety of samplers, as listed in the table. Sample weight and depth below surface are noted. Weight is measured using a sping scale, and small stuff sack for sample. Sample depth is taken with a tape measurer in pits, or measured along core, and in core hole.'

In [11]:
#Save name of table
table_name=row.table

# Add Metadata for Numeric Columns

In [12]:
#read in metadata tab in excel workbook for numeric columns
met_tab=pd.read_excel(met_tab_pth, sheetname='columns_numeric')

In [13]:
met_tab

,column,units,description,blank_description,uncertainty
0,sample_weight,grams,Weight of snow sample,Sample lost or not measurable; very rare.,5.0
1,sample_bottom_depth,centimeters,Bottom depth of snow sample,NaN,1.0
2,avg_core_length,centimeters,"Average of 3 measurements, on 3 sides of core","If pit measurement, no value",0.5
3,avg_core_diam,centimeters,"Average of 4 measurements, on 4 opposing sides...","If pit measurement, no value",0.2
4,volume,cubic centimeters,"Sample volume, from either sampler, or calcula...",NaN,5.0


### Add table-level entity information:

In [14]:
#Input data in example is for a single table only
metd.metadata.eainfo.detailed.enttyp.enttypl.text=table_name
metd.metadata.eainfo.detailed.enttyp.enttypd.text='table' #in this system, all inputs are tables
metd.metadata.eainfo.detailed.enttyp.enttypds.text='Producer defined' #only change if proprietary format; not the case here

In [15]:
metd.metadata.eainfo.detailed.enttyp

<enttyp>
  <enttypl>GlacierNameYYYY.MM.DD PitCore_SiteLabel.csv</enttypl>
  <enttypd>table</enttypd>
  <enttypds>Producer defined</enttypds>
</enttyp>

### Add column-level attribute information

In [16]:
#Read in a blank node section for numeric attributes
attr_n=XMLNode(open(attr_numeric_pth, 'r').read()) #Must read in text file as string to convert to node

In [17]:
attr_n

<attr>
  <attrlabl>

  </attrlabl>
  <attrdef>

  </attrdef>
  <attrdefs>

  </attrdefs>
  <attrdomv>
    <rdom>
      <rdommin>

      </rdommin>
      <rdommax>

      </rdommax>
      <attrunit>

      </attrunit>
      <attrmres>

      </attrmres>
    </rdom>
  </attrdomv>
</attr>

#### Add information for each numeric data type (each row below)

In [18]:
met_tab

,column,units,description,blank_description,uncertainty
0,sample_weight,grams,Weight of snow sample,Sample lost or not measurable; very rare.,5.0
1,sample_bottom_depth,centimeters,Bottom depth of snow sample,NaN,1.0
2,avg_core_length,centimeters,"Average of 3 measurements, on 3 sides of core","If pit measurement, no value",0.5
3,avg_core_diam,centimeters,"Average of 4 measurements, on 4 opposing sides...","If pit measurement, no value",0.2
4,volume,cubic centimeters,"Sample volume, from either sampler, or calcula...",NaN,5.0


In [19]:
#Loop through all rows in table; add metadata to appropriate nodes
for rwnum in met_tab.index:
    row=met_tab.iloc[rwnum:rwnum+1:]
    metd.metadata.eainfo.detailed.add_child(attr_n)#add new attribute section to the metadata
    #Add column title
    if rwnum==0:
        metd.metadata.eainfo.detailed.attr.attrlabl.text=row.iloc[0]['column']
        metd.metadata.eainfo.detailed.attr.attrdef.text=row.iloc[0]['description']
        metd.metadata.eainfo.detailed.attr.attrdomv.rdom.attrunit.text=row.iloc[0]['units']
        metd.metadata.eainfo.detailed.attr.attrdefs.text='Producer Defined'
    else:
        metd.metadata.eainfo.detailed.attr[rwnum].attrlabl.text=row.iloc[0]['column']
        metd.metadata.eainfo.detailed.attr[rwnum].attrdef.text=row.iloc[0]['description']
        metd.metadata.eainfo.detailed.attr[rwnum].attrdomv.rdom.attrunit.text=row.iloc[0]['units']
        metd.metadata.eainfo.detailed.attr[rwnum].attrdefs.text='Producer Defined'

# Add factor/ categorical columns

In [20]:
#import metadta on factor-type columns
met_tab=pd.read_excel(met_tab_pth, sheetname='columns_factor')
#import template for factor XML
#Read in a blank node section for numeric attributes
attr_factor=XMLNode(open(attr_factor_path, 'r').read()) #Must read in text file as string to convert to node
factor_details=XMLNode(open(category_details_path, 'r').read())

In [21]:
met_tab

,column,description,levels,blank_description
0,sampler,Type of density sampler used,"wedge, Felix, Kovacs, tube",NaN


In [22]:
#Loop through all rows in categorical/factor table; add metadata to appropriate nodes
for rwnum in met_tab.index:
    row=met_tab.iloc[rwnum:rwnum+1:]
    metd.metadata.eainfo.detailed.add_child(attr_factor)#add new attribute section to the metadata
    #already have added numeric columns so can add as below with list indexing
    metd.metadata.eainfo.detailed.attr[-1].attrlabl.text=row.iloc[0]['column']
    metd.metadata.eainfo.detailed.attr[-1].attrdef.text=row.iloc[0]['description']
    levels=row.iloc[0]['levels'].split(",")
    count=-1
    for xx in levels:
        count=count+1
        xx=xx.strip()
        print(xx)
        metd.metadata.eainfo.detailed.attr[-1].add_child(factor_details) #add details on factor section
        if count==0 and rwnum==0:
            metd.metadata.eainfo.detailed.attr[-1].attrdomv.edom.edomv.text=xx
        else: 
            metd.metadata.eainfo.detailed.attr[-1].attrdomv[-1].edom.edomv.text=xx

wedge
Felix
Kovacs
tube


In [23]:
#Add date metadata was created (today)
today=datetime.datetime.today().strftime('%Y%m%d')
metd.metadata.metainfo.metd.text=today

# Done! 
## At least with the automated part here
#### Now, open with metadata wizard, and perfect for distribution.

In [24]:
out_path=r"C:\Users\ehbaker\Documents\XML_Work" #set the path for your output XML docs
metd.save(fname=os.path.join(out_path,"Improved_metadata.xml"))
metd.metadata

<metadata>
  <idinfo>
    <citation>
      <citeinfo>
        <origin>Emily H. Baker</origin>
        <pubdate>

        </pubdate>
        <title>

        </title>
        <geoform>

        </geoform>
        <pubinfo>
          <pubplace>Anchorage, AK</pubplace>
          <publish>U.S. Geological Survey, Alaska Science Center</publish>
        </pubinfo>
        <onlink>https://doi.org/10.5066/xxxxxxxx</onlink>
        <lworkcit>
          <citeinfo>
            <origin>

            </origin>
            <pubdate>

            </pubdate>
            <title>

            </title>
            <geoform>

            </geoform>
          </citeinfo>
        </lworkcit>
        <origin>Christopher J. McNeil</origin>
        <origin>Louis C. Sass</origin>
        <origin>Shad O'Neel</origin>
      </citeinfo>
    </citation>
    <descript>
      <abstract>Raw snow pit and snow core measurements of sample weight and volume, at specified depth from the surface</abstract>
      <purpose>

